# Created by Sunny You on September 1, 2023

This script adds in the future of each storm's time of landfall to the model's training data.

In [1]:
import numpy as np
import pandas as pd
import os
import re

In [2]:
os.chdir(os.path.dirname(os.getcwd()))
print(os.getcwd())

/Users/sunnyyou/Documents/Real_Time_HAI/HRICNN


In [3]:
data = pd.read_csv('BRTK_2000to2019_IMERG_SHIPS-RII.csv')
data['ID'] = data.GIS_ID.str[:10]
data = data[["GIS_ID", "JULDAY","LAND_OCEAN","ID"]]
data

,GIS_ID,JULDAY,LAND_OCEAN,ID
0,ATL_200001_TD_2000060718,2451703.25,Ocean,ATL_200001
1,ATL_200001_TD_2000060800,2451703.50,Ocean,ATL_200001
2,ATL_200001_TD_2000060806,2451703.75,Ocean,ATL_200001
3,ATL_200001_TD_2000060812,2451704.00,Ocean,ATL_200001
4,ATL_200002_TD_2000062300,2451718.50,Ocean,ATL_200002
...,...,...,...,...
38721,SPA_201927_TD_2019051512,2458619.00,Ocean,SPA_201927
38722,SPA_201927_TD_2019051518,2458619.25,Ocean,SPA_201927
38723,SPA_201927_TD_2019051600,2458619.50,Ocean,SPA_201927
38724,SPA_201927_TD_2019051618,2458620.25,Ocean,SPA_201927


In [4]:
s = data[data.ID == "ATL_200001"]
s

,GIS_ID,JULDAY,LAND_OCEAN,ID
0,ATL_200001_TD_2000060718,2451703.25,Ocean,ATL_200001
1,ATL_200001_TD_2000060800,2451703.50,Ocean,ATL_200001
2,ATL_200001_TD_2000060806,2451703.75,Ocean,ATL_200001
3,ATL_200001_TD_2000060812,2451704.00,Ocean,ATL_200001


In [5]:
final_df = pd.DataFrame(columns = ["GIS_ID", "LOP06", "LOP12", "LOP18", "LOP24"])

i=0
for i in range(data.shape[0]):
    if i%1000 == 0:
        print(i)
    row = data.loc[i]
    cur_time = row.JULDAY
    temp = data[data.ID == row.ID]
    p6_time = cur_time + 0.25
    p12_time = cur_time + 0.5
    p18_time = cur_time + 0.75
    p24_time = cur_time + 1.0

    p6 = temp[temp.JULDAY == p6_time]
    p12 = temp[temp.JULDAY == p12_time]
    p18 = temp[temp.JULDAY == p18_time]
    p24 = temp[temp.JULDAY == p24_time]

    if p6.shape[0] == 0:
        p6 = "-999"
    else: 
        p6 = p6.LAND_OCEAN.iloc[0]

    if p12.shape[0] == 0:
        p12 = "-999"
    else: 
        p12 = p12.LAND_OCEAN.iloc[0]

    if p18.shape[0] == 0:
        p18 = "-999"
    else: 
        p18 = p18.LAND_OCEAN.iloc[0]

    if p24.shape[0] == 0:
        p24 = "-999"
    else: 
        p24 = p24.LAND_OCEAN.iloc[0]

    final_df.loc[i] = [row.GIS_ID, p6, p12, p18, p24]

final_df


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000


,GIS_ID,LOP06,LOP12,LOP18,LOP24
0,ATL_200001_TD_2000060718,Ocean,Ocean,Ocean,-999
1,ATL_200001_TD_2000060800,Ocean,Ocean,-999,-999
2,ATL_200001_TD_2000060806,Ocean,-999,-999,-999
3,ATL_200001_TD_2000060812,-999,-999,-999,-999
4,ATL_200002_TD_2000062300,Ocean,Ocean,Ocean,Ocean
...,...,...,...,...,...
38721,SPA_201927_TD_2019051512,Ocean,Ocean,-999,-999
38722,SPA_201927_TD_2019051518,Ocean,-999,-999,Ocean
38723,SPA_201927_TD_2019051600,-999,-999,Ocean,-999
38724,SPA_201927_TD_2019051618,-999,-999,-999,-999


In [6]:
f = final_df
f["LOP12"] = np.where(f.LOP06 != "Ocean", f.LOP06, f.LOP12)
f["LOP18"] = np.where(f.LOP12 != "Ocean", f.LOP12, f.LOP18)
f["LOP24"] = np.where(f.LOP18 != "Ocean", f.LOP18, f.LOP24)

In [7]:
f

,GIS_ID,LOP06,LOP12,LOP18,LOP24
0,ATL_200001_TD_2000060718,Ocean,Ocean,Ocean,-999
1,ATL_200001_TD_2000060800,Ocean,Ocean,-999,-999
2,ATL_200001_TD_2000060806,Ocean,-999,-999,-999
3,ATL_200001_TD_2000060812,-999,-999,-999,-999
4,ATL_200002_TD_2000062300,Ocean,Ocean,Ocean,Ocean
...,...,...,...,...,...
38721,SPA_201927_TD_2019051512,Ocean,Ocean,-999,-999
38722,SPA_201927_TD_2019051518,Ocean,-999,-999,-999
38723,SPA_201927_TD_2019051600,-999,-999,-999,-999
38724,SPA_201927_TD_2019051618,-999,-999,-999,-999


In [8]:
original_data = pd.read_csv('BRTK_2000to2019_IMERG_SHIPS-RII.csv')
original_data

,GIS_ID,BASIN,DATE,JULDAY,LATITUDE,LONGITUDE,MSLP,STORM_NAME,STORM_ID,INT_CAT,...,SHIPS_IR00_12h,SHIPS_DTL_00h,SHIPS_DTL_06h,SHIPS_DTL_12h,SHIPS_DTL_18h,SHIPS_DTL_24h,SHIPS_DTL_30h,SHIPS_DTL_36h,SHIPS_DTL_42h,SHIPS_DTL_48h
0,ATL_200001_TD_2000060718,ATL,2000060718,2451703.25,21.000000,-93.000000,1008,NOT_NAMED,200001,TD,...,-999,272,237,279,314,-999,-999,-999,-999,-999
1,ATL_200001_TD_2000060800,ATL,2000060800,2451703.50,20.900000,-92.800003,1009,NOT_NAMED,200001,TD,...,118,252,268,303,-999,-999,-999,-999,-999,-999
2,ATL_200001_TD_2000060806,ATL,2000060806,2451703.75,20.700001,-93.099998,1010,NOT_NAMED,200001,TD,...,-999,242,282,-999,-999,-999,-999,-999,-999,-999
3,ATL_200001_TD_2000060812,ATL,2000060812,2451704.00,20.799999,-93.500000,1010,NOT_NAMED,200001,TD,...,-999,262,-999,-999,-999,-999,-999,-999,-999,-999
4,ATL_200002_TD_2000062300,ATL,2000062300,2451718.50,9.500000,-19.799999,1010,NOT_NAMED,200002,TD,...,-999,444,530,656,844,1037,1220,1358,1408,1683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38721,SPA_201927_TD_2019051512,SPA,2019051512,2458619.00,-12.200000,141.000000,1009,ANN,201927,TD,...,269,78,235,228,-999,-999,200,216,-999,-999
38722,SPA_201927_TD_2019051518,SPA,2019051518,2458619.25,-11.800000,139.600006,1009,ANN,201927,TD,...,224,235,228,-999,-999,200,216,-999,-999,-999
38723,SPA_201927_TD_2019051600,SPA,2019051600,2458619.50,-11.200000,138.500000,1008,ANN,201927,TD,...,249,228,-999,-999,200,216,-999,-999,-999,-999
38724,SPA_201927_TD_2019051618,SPA,2019051618,2458620.25,-10.300000,135.199997,1009,ANN,201927,TD,...,241,200,216,-999,-999,-999,-999,-999,-999,-999


In [9]:
original_data = original_data.merge(f, on = "GIS_ID")
original_data

,GIS_ID,BASIN,DATE,JULDAY,LATITUDE,LONGITUDE,MSLP,STORM_NAME,STORM_ID,INT_CAT,...,SHIPS_DTL_18h,SHIPS_DTL_24h,SHIPS_DTL_30h,SHIPS_DTL_36h,SHIPS_DTL_42h,SHIPS_DTL_48h,LOP06,LOP12,LOP18,LOP24
0,ATL_200001_TD_2000060718,ATL,2000060718,2451703.25,21.000000,-93.000000,1008,NOT_NAMED,200001,TD,...,314,-999,-999,-999,-999,-999,Ocean,Ocean,Ocean,-999
1,ATL_200001_TD_2000060800,ATL,2000060800,2451703.50,20.900000,-92.800003,1009,NOT_NAMED,200001,TD,...,-999,-999,-999,-999,-999,-999,Ocean,Ocean,-999,-999
2,ATL_200001_TD_2000060806,ATL,2000060806,2451703.75,20.700001,-93.099998,1010,NOT_NAMED,200001,TD,...,-999,-999,-999,-999,-999,-999,Ocean,-999,-999,-999
3,ATL_200001_TD_2000060812,ATL,2000060812,2451704.00,20.799999,-93.500000,1010,NOT_NAMED,200001,TD,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,ATL_200002_TD_2000062300,ATL,2000062300,2451718.50,9.500000,-19.799999,1010,NOT_NAMED,200002,TD,...,844,1037,1220,1358,1408,1683,Ocean,Ocean,Ocean,Ocean
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38721,SPA_201927_TD_2019051512,SPA,2019051512,2458619.00,-12.200000,141.000000,1009,ANN,201927,TD,...,-999,-999,200,216,-999,-999,Ocean,Ocean,-999,-999
38722,SPA_201927_TD_2019051518,SPA,2019051518,2458619.25,-11.800000,139.600006,1009,ANN,201927,TD,...,-999,200,216,-999,-999,-999,Ocean,-999,-999,-999
38723,SPA_201927_TD_2019051600,SPA,2019051600,2458619.50,-11.200000,138.500000,1008,ANN,201927,TD,...,200,216,-999,-999,-999,-999,-999,-999,-999,-999
38724,SPA_201927_TD_2019051618,SPA,2019051618,2458620.25,-10.300000,135.199997,1009,ANN,201927,TD,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [11]:
original_data.to_csv("IMERG/Land_Ocean_Futures.csv")